In [4]:
import numpy as np
import cv2 as cv
import json

In [6]:
def execute(src, out, minArea=5):
    img = cv.imread(src, 0)
    contours, hierarchy = cv.findContours(img, cv.RETR_TREE,  cv.CHAIN_APPROX_NONE)
    # Sort by larger areas
    contoursFeatures = []
    cnts = sorted(contours, key=cv.contourArea, reverse=True)
    for i in range(0, len(cnts)):
        if cv.contourArea(cnts[i]) >= minArea:
            features = {}
            M = cv.moments(cnts[i])
            features.centroid = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            features.arcLength = cv.arcLength(cnts[i], True)
            features.area = cv.contourArea(cnts[i])
            try:
                # Calculate eccentricity
                (x, y), (MA, ma), angle = cv.fitEllipse(cnts[i])
                a = ma/2
                b = MA/2

                if (a > b):
                    eccentricity = sqrt(pow(a, 2)-pow(b, 2))
                    eccentricity = round(eccentricity/a, 2)
                else:
                    eccentricity = sqrt(pow(b, 2)-pow(a, 2))
                    eccentricity = round(eccentricity/b, 2)

                features.eccentricity = eccentricity
            except:
                features.eccentricity = -1
            contoursFeatures.append(features)
    f = open(out)
    f.write(json(contoursFeatures))

In [7]:
execute('./dl1.png', 'contours.json', 5)

NameError: name 'cv2' is not defined